# Demo of KG construction for reliability applications
This notebook is intended to show the steps required to construct the KG that include
- MBSE representation of the considered system
- Anomalies detected from numeric data
- Textual elements processed by TLP methods 

This notebook requires that:
* Neo4j desktop is installed in your machine
* A neo4j project is initialized with the corresponding Graph DBMS such that the bolt port is available
* the csv files are located in the correspoding folder; on MAC this can be found in this location:
  /"User"/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/"dbms-ID"/import

In [73]:
# Source: https://github.com/cj2001/bite_sized_data_science/tree/main
from neo4j import GraphDatabase

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response
    
uri = "to be added"
pwd = "to be added"

conn = Neo4jConnection(uri=uri, user='neo4j', pwd=pwd)

## Clean all

In [75]:
result = conn.query('MATCH (n) DETACH DELETE n;')
print(result)

[]


# Load MBSE nodes and edges

In [76]:
query2 = """
LOAD CSV WITH HEADERS FROM 'file:////test_nodes.csv' AS row 
MERGE (e:MBSE {nodeId: row.nodeId, label: row.label, ID: row.ID, type: row.type}) return row """

result = conn.query(query2)
print(result)

query3 = """
LOAD CSV WITH HEADERS FROM 'file:///test_edges.csv' AS row
MATCH (e:MBSE {nodeId: row.sourceNodeId})
MATCH (c:MBSE {nodeId: row.targetNodeId})
CREATE (e)-[:MBSE_link {prop:row.type}]->(c) """

result = conn.query(query3)
print(result)

[<Record row={'nodeId': '0', 'ID': 'C3', 'label': 'C3', 'type': 'entity_emb'}>, <Record row={'nodeId': '1', 'ID': 'cond1', 'label': 'condenser', 'type': 'entity'}>, <Record row={'nodeId': '2', 'ID': 'I_1PC9PKV4BTGV5_BMXNGD2MYES4D', 'label': 'pipe', 'type': 'LML_link'}>, <Record row={'nodeId': '3', 'ID': 'I_9YG45MX0QMK91_80WEVYKF2ZK18', 'label': 'pipe', 'type': 'LML_link'}>, <Record row={'nodeId': '4', 'ID': 'V2', 'label': 'V2', 'type': 'entity_emb'}>, <Record row={'nodeId': '5', 'ID': 'V3', 'label': 'V3', 'type': 'entity_emb'}>, <Record row={'nodeId': '6', 'ID': 'S3', 'label': 'level sensor', 'type': 'entity_emb'}>, <Record row={'nodeId': '7', 'ID': 'None', 'label': 'forebay', 'type': 'entity'}>, <Record row={'nodeId': '8', 'ID': 'body', 'label': 'OPM_pump', 'type': 'MBSE_linked_ent'}>, <Record row={'nodeId': '9', 'ID': 'PM1', 'label': 'pump', 'type': 'entity'}>, <Record row={'nodeId': '10', 'ID': 'I_2J6B8DXJ2WH0N_AR6M5HTA4X3SZ', 'label': 'pipe', 'type': 'LML_link'}>, <Record row={'nod

## Load monitoring vars and their link to MBSE nodes

In [77]:
query4 = """LOAD CSV WITH HEADERS FROM 'file:///test_monit_vars.csv' AS row
MERGE (v:monitor_var {nodeId: row.varID})
return row """
result = conn.query(query4)
print(result)

query5 = """LOAD CSV WITH HEADERS FROM 'file:///test_monit_vars.csv' AS row
MATCH (v:monitor_var {nodeId: row.varID})
MATCH (e:MBSE {nodeId: row.MBSE_ID})
CREATE (v)-[:monitoring]->(e) """
result = conn.query(query5)
print(result)

[<Record row={'varID': 'x_1', 'MBSE_ID': '9'}>, <Record row={'varID': 'x_2', 'MBSE_ID': '12'}>, <Record row={'varID': 'T_cond', 'MBSE_ID': '1'}>]
[]


## Load anomaly detection methods and their link to monitoring vars

In [78]:
query6 = """LOAD CSV WITH HEADERS FROM 'file:///test_AD.csv' AS row
MERGE (ad:anomaly_detect {nodeId: row.AD_ID, type: row.type})
return row"""
result = conn.query(query6)
print(result)

query7 = """LOAD CSV WITH HEADERS FROM 'file:///test_AD.csv' AS row
MATCH (ad:anomaly_detect {nodeId: row.AD_ID})
MATCH (v:monitor_var {nodeId: row.var_ID})
CREATE (ad)-[:input_from]->(v)"""
result = conn.query(query7)
print(result)

[<Record row={'type': 'matrix_profile', 'var_ID': 'x_1', 'AD_ID': '1'}>, <Record row={'type': 'matrix_profile', 'var_ID': 'x_2', 'AD_ID': '2'}>, <Record row={'type': 'matrix_profile', 'var_ID': 'T_cond', 'AD_ID': '3'}>, <Record row={'type': 'matrix_profile_ND', 'var_ID': 'x_1', 'AD_ID': '4'}>, <Record row={'type': 'matrix_profile_ND', 'var_ID': 'x_2', 'AD_ID': '4'}>]
[]


## Load detected anomalies and link to anomaly detection methods

In [79]:
query8 = """LOAD CSV WITH HEADERS FROM 'file:///test_anomalies.csv' AS row
MERGE (a:anomaly {nodeId: row.anom_ID, time_initial: row.t_in, time_final: row.t_fin})
return row """
result = conn.query(query8)
print(result)

query9 = """LOAD CSV WITH HEADERS FROM 'file:///test_anomalies.csv' AS row
MATCH (a:anomaly {nodeId: row.anom_ID})
MATCH (ad:anomaly_detect {nodeId: row.AD_ID})
CREATE (a)-[:detected_by]->(ad) """
result = conn.query(query9)
print(result)

[<Record row={'t_in': '1', 't_fin': '2', 'anom_ID': '1', 'AD_ID': '1'}>, <Record row={'t_in': '5.5', 't_fin': '6', 'anom_ID': '2', 'AD_ID': '2'}>, <Record row={'t_in': '8', 't_fin': '9.2', 'anom_ID': '3', 'AD_ID': '3'}>, <Record row={'t_in': '10', 't_fin': '10.5', 'anom_ID': '4', 'AD_ID': '4'}>, <Record row={'t_in': '6.8', 't_fin': '7.1', 'anom_ID': '5', 'AD_ID': '3'}>]
[]


In [80]:
query10 = """MATCH (n) RETURN n """
result = conn.query(query10)
print(result)

[<Record n=<Node element_id='4:e3d123e1-70ab-499a-8eca-ab1484677aac:8' labels=frozenset({'MBSE'}) properties={'label': 'C3', 'ID': 'C3', 'type': 'entity_emb', 'nodeId': '0'}>>, <Record n=<Node element_id='4:e3d123e1-70ab-499a-8eca-ab1484677aac:9' labels=frozenset({'MBSE'}) properties={'ID': 'cond1', 'label': 'condenser', 'type': 'entity', 'nodeId': '1'}>>, <Record n=<Node element_id='4:e3d123e1-70ab-499a-8eca-ab1484677aac:10' labels=frozenset({'MBSE'}) properties={'ID': 'I_1PC9PKV4BTGV5_BMXNGD2MYES4D', 'label': 'pipe', 'type': 'LML_link', 'nodeId': '2'}>>, <Record n=<Node element_id='4:e3d123e1-70ab-499a-8eca-ab1484677aac:11' labels=frozenset({'MBSE'}) properties={'ID': 'I_9YG45MX0QMK91_80WEVYKF2ZK18', 'label': 'pipe', 'type': 'LML_link', 'nodeId': '3'}>>, <Record n=<Node element_id='4:e3d123e1-70ab-499a-8eca-ab1484677aac:12' labels=frozenset({'MBSE'}) properties={'label': 'V2', 'ID': 'V2', 'type': 'entity_emb', 'nodeId': '4'}>>, <Record n=<Node element_id='4:e3d123e1-70ab-499a-8eca-ab